<a href="https://colab.research.google.com/github/chizhenn/DS-ML-Projects/blob/main/Algorithm_Design_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Import dataset**

In [ ]:
import io
import requests
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
df = "/content/gdrive/MyDrive/Moira_Market_Items.csv"
df = pd.read_csv (df, header=0)

##**Understand the dataset**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Stall ID       350 non-null    object 
 1   Item Name      350 non-null    object 
 2   Price          350 non-null    int64  
 3   Durability     350 non-null    int64  
 4   Compatibility  350 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 13.8+ KB


*   There are 5 features with no missing values. Stall ID and Item Name act as identifiers.

*   Compatibility is how well the item works with the ship, which is the most important feature when selecting an item.

*   Price and durability are essential factors and should be evaluated according to user requirements.




In [ ]:
df.head()

,Stall ID,Item Name,Price,Durability,Compatibility
0,STALL-004,Nano Filter,434,53,0.90
1,STALL-186,Solar Filter,1031,62,0.65
2,STALL-153,Solar Core,394,98,0.99
3,STALL-047,Quantum Core,854,51,0.80
4,STALL-038,Cyber Relay,1494,54,0.79


##**Check duplication**

In [ ]:
# Identify duplicate rows based on 'Stall ID' and 'Item Name'
duplicates = df.duplicated(subset=['Stall ID', 'Item Name'], keep=False)

# Count number of duplicate entries
num_duplicates = duplicates.sum()
duplicate_rows = df[duplicates]

print(f"Number of duplicate rows (Stall ID + Item Name): {num_duplicates}")
print(duplicate_rows)

Number of duplicate rows (Stall ID + Item Name): 12
      Stall ID       Item Name  Price  Durability  Compatibility
31   STALL-035   Quantum Relay    603          85           0.86
47   STALL-179     Solar Drive    386          52           0.77
51   STALL-028   Plasma Matrix    691          72           0.78
96   STALL-158        Ion Core    792          64           0.90
137  STALL-179     Solar Drive    843          63           0.79
175  STALL-035   Quantum Relay    554          54           0.62
235  STALL-161     Nano Matrix    886          99           0.73
245  STALL-158        Ion Core    205          98           0.94
254  STALL-161     Nano Matrix   1474          84           0.73
268  STALL-002  Neutron Filter   1499          50           0.79
281  STALL-028   Plasma Matrix   1083          71           0.99
294  STALL-002  Neutron Filter   1183          72           0.98


In [ ]:
# Identify duplicate rows based on 'Stall ID' and 'Item Name'
duplicates = df.duplicated(subset=['Stall ID', 'Item Name','Compatibility'], keep=False)

# Count number of duplicate entries
num_duplicates = duplicates.sum()
duplicate_rows = df[duplicates]

print(f"Number of duplicate rows (Stall ID + Item Name): {num_duplicates}")
print(duplicate_rows)

Number of duplicate rows (Stall ID + Item Name): 2
      Stall ID    Item Name  Price  Durability  Compatibility
235  STALL-161  Nano Matrix    886          99           0.73
254  STALL-161  Nano Matrix   1474          84           0.73


*   Based on the results, it was found that the same item from the same stall may have different versions. Some may share the same compatibility, but they can still be differentiated by price and durability.






##**Constraints Given**

- Maximum Price : Items must cost under 800 credits .
- Minimum Durability : Items must have a durability rating above 85 .
- Time Constraint : The market resets every 20 minutes, so we need to quickly filter and prioritize viable options .


Based on the data analysis, we have developed two logic algorithms:






##**Algorithm 1 - Filter & Sort**

In [ ]:
# Apply filters
filtered_df = df[
    (df['Price'] < 800) &
    (df['Durability'] > 85) &
    (df['Compatibility'] >= 0.8)
].copy()

# Add Durability / Price ratio
filtered_df['Durability/Price'] = filtered_df['Durability'] / filtered_df['Price']

# Sort by Compatibility then Durability per Price
sorted_df = filtered_df.sort_values(
    by=['Compatibility', 'Durability/Price'],
    ascending=[False, False]
)

# Add Rank column
# Optional
sorted_df['Rank'] = range(1, len(sorted_df) + 1)

# Reorder and display relevant columns
result = sorted_df[[
    'Rank', 'Stall ID', 'Item Name', 'Price', 'Durability',
    'Compatibility', 'Durability/Price'
]]

print(result.to_string(index=False))

 Rank  Stall ID      Item Name  Price  Durability  Compatibility  Durability/Price
    1 STALL-153     Solar Core    394          98           0.99          0.248731
    2 STALL-076     Hyper Lens    397          98           0.98          0.246851
    3 STALL-181   Plasma Relay    629          87           0.96          0.138315
    4 STALL-158       Ion Core    205          98           0.94          0.478049
    5 STALL-151     Ion Filter    625          93           0.94          0.148800
    6 STALL-167   Neutron Lens    747          90           0.93          0.120482
    7 STALL-190   Neutron Lens    113          93           0.92          0.823009
    8 STALL-182    Hyper Drive    518          95           0.91          0.183398
    9 STALL-029    Plasma Core    658          97           0.90          0.147416
   10 STALL-094    Solar Drive    763          93           0.90          0.121887
   11 STALL-143 Neutron Shield    285          93           0.87          0.326316


In short, we filter the items based on:

- Price < 800

- Durability > 85

- Compatibility ≥ 0.8 (assumed threshold)

Then, sort and rank the filtered results by:

- Primary: Compatibility (descending)

- Secondary: Durability/Price ratio (descending)

This ensures:

- We pick items that fit the ship best first.

- Then, among those, we get the best durability per credit spent.

**Note:**

We define Threshold = 0.8, which is a practical, safe, and realistic cutoff that balances performance, safety, and availability in volatile market environment

##**Algorithm 2 - Heap-based**

In [ ]:
import heapq

def get_ranked_modules(df, top_n=10):
    pq = []

    # Filter, score, and maintain top_n in heap
    for _, row in df.iterrows():
        price = row['Price']
        durability = row['Durability']
        compatibility = row['Compatibility']

        if price >= 800 or durability <= 85 or compatibility < 0.8:
            continue

        score = compatibility * 1000 - price + durability / 10
        entry = (score, row['Stall ID'], row['Item Name'], price, durability, compatibility)

        if len(pq) < top_n:
            heapq.heappush(pq, entry)
        else:
            # keep highest scores only
            heapq.heappushpop(pq, entry)

    # Sort results in descending score order
    results = []
    rank = 1
    for item in heapq.nlargest(top_n, pq):
        results.append({
            'Ranking': rank,
            'Stall ID': item[1],
            'Item Name': item[2],
            'Price': item[3],
            'Durability': item[4],
            'Compatibility': item[5],
            'Score': item[0]
        })
        rank += 1

    return pd.DataFrame(results)

ranked_df = get_ranked_modules(df, top_n=10)
print(ranked_df.to_string(index=False))


 Ranking  Stall ID      Item Name  Price  Durability  Compatibility  Score
       1 STALL-190   Neutron Lens    113          93           0.92  816.3
       2 STALL-158       Ion Core    205          98           0.94  744.8
       3 STALL-153     Solar Core    394          98           0.99  605.8
       4 STALL-143 Neutron Shield    285          93           0.87  594.3
       5 STALL-076     Hyper Lens    397          98           0.98  592.8
       6 STALL-182    Hyper Drive    518          95           0.91  401.5
       7 STALL-181   Plasma Relay    629          87           0.96  339.7
       8 STALL-151     Ion Filter    625          93           0.94  324.3
       9 STALL-029    Plasma Core    658          97           0.90  251.7
      10 STALL-167   Neutron Lens    747          90           0.93  192.0


In short, we filter the items based on:

- Price < 800

- Durability > 85

- Compatibility ≥ 0.8 (assumed threshold)

Then, we use a priority queue (heap) to rank (top_n) the filtered results by:

- Score = Compatibility × 1000 – Price + Durability ÷ 10


**Note:**
- Python’s `heapq` only creates min-heaps by default, meaning it always pops the smallest item first, which is not ideal when we want the best or highest-ranked item.

- To simulate a max-heap, we use a trick: store the negative version of the score (e.g., if the score is 600, store it as -600).

- This works because the highest positive score becomes the smallest (most negative) number, so when `heapq` pops the smallest item, it's actually giving us the best module based on our ranking score.

**Note:**

How we define Score = Compatibility × 1000 – Price + Durability ÷ 10

- Compatibility × 1000

Compatibility values are decimals (like 0.99), and prices are in the hundreds, so multiplying compatibility by 1000 makes it a major factor in the score, to ensure only well-fitting modules are considered.

- Price

We want to favor cheaper modules because lower price means better value, so subtracting the price ensures that, among equally compatible items, the cheaper one ranks higher.

- Durability ÷ 10

While durability is important for long-term use, it’s not as critical as compatibility or cost, so dividing it by 10 gives it a smaller boost, which rewards more durable items without overpowering the other factors.


##**Conclusion**

Heap-based method is chosen, because:

- Heap-based method keeps top_n elements in O(n log top_n) (top_n is small compared to n), while full sort takes O(n log n).
- Time grows slowly even with large datasets as long as top_n is small or fixed.
- Heap-based method is suitable for datasets in real-time or streaming where results are updated incrementally.
